In [20]:
import pandas as pd
import numpy as np
from tensorflow.python.ops import nn_ops, gen_nn_ops
import tensorflow as tf

In [38]:
class CNN_MC_dropout:
    def __init__(self, std=0.01 , batch_size=64,width=500, height =1, input_channel=3, l_rate =1e-6,reuse = False):
        self.std=std
        self.batch_size=batch_size
        self.height = height
        self.width = width
        self.input_channel = input_channel
        self.l_rate = l_rate

        with tf.name_scope('Classifier'):
            self.y = tf.placeholder(tf.float32, [None, 3], name='y')
            self.x = tf.placeholder(tf.float32, [None, self.height,self.width,self.input_channel], name='x')
            self.dropout_bool = tf.placeholder(tf.bool, [None,1])
            self.keep_prob = tf.placeholder(tf.float32)

        self.logits = self.build_model()

        # Define loss and optimizer, minimize the squared error
        self.cross_entropy = cross_entropy = tf.nn.softmax_cross_entropy_with_logits_v2(labels=self.y, logits=self.logits)
        self.cost =tf.reduce_mean(self.cross_entropy)
        self.optimizer = tf.train.AdamOptimizer(self.l_rate).minimize(self.cost)

        self.correct_pred = tf.equal(tf.round(self.prediction),self.y)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_pred, tf.float32))

        init = tf.global_variables_initializer()

        # Launch the session
        self.session_conf = tf.ConfigProto()
        self.session_conf.gpu_options.allow_growth = True
        self.sess = tf.InteractiveSession(config=self.session_conf)
        self.sess.run(init)

        self.saver = tf.train.Saver(max_to_keep=None)

    def build_model(self):
        with tf.variable_scope('layer0'):
            #b, h, w, c
            self.input = self.x
        # Convolutional Layer #1 and Pooling Layer #1
        with tf.variable_scope('layer1'):
            self.conv1 = tf.layers.conv2d(self.input, 128, [8,1], padding='SAME')
            self.batch1 = tf.layers.batch_normalization(self.conv1)
            self.relu1 = tf.nn.relu(self.batch1)
            self.dropout1 = tf.layers.dropout(self.relu1, self.keep_prob,self.dropout_bool)

        # Convolutional Layer #1 and Pooling Layer #2
        with tf.variable_scope('layer2'):
            self.conv2 = tf.layers.conv2d(self.dropout1, 256, [5,1], padding='SAME')
            self.batch2 = tf.layers.batch_normalization(self.conv2)
            self.relu2 = tf.nn.relu(self.batch2)
            self.dropout2 = tf.layers.dropout(self.relu2, self.keep_prob,self.dropout_bool)

        # Convolutional Layer #1 and Pooling Layer #3
        with tf.variable_scope('layer3'):
            self.conv3 = tf.layers.conv2d(self.dropout2, 128, [3,1], padding='SAME')
            self.batch3 = tf.layers.batch_normalization(self.conv3)
            self.relu3 = tf.nn.relu(self.batch3)

        # Dense Layer with Relu
        with tf.variable_scope('layer4'):
            #Global Average Pooling
            self.GAP = tf.reduce_mean(self.relu3, axis=[1,2])# b,h,w,c
            self.logits = tf.layers.dense(self.GAP,3)
            self.prediction = tf.nn.softmax(self.logits)
            
        return self.logits


    def train(self, data, target, keep_prob,dropout_bool):
        opt, cost ,acc = self.sess.run((self.optimizer, self.cost, self.accuracy ), 
                             feed_dict={self.y: target,
                                        self.x: data,
                                       self.keep_prob: keep_prob,
                                       self.dropout_bool : dropout_bool})
        return cost,acc

    def test(self, data, target, keep_prob,dropout_bool):
        cost,acc = self.sess.run((self.cost,self.accuracy),
                             feed_dict={self.y: target,
                                        self.x: data,
                                       self.keep_prob: keep_prob,
                                       self.dropout_bool : dropout_bool})
        return cost,acc

    def get_last_conv_output(self, data, keep_prob):
        relu3 =  self.sess.run((self.relu3), 
                             feed_dict={self.x: data,
                                       self.keep_prob: keep_prob})
        return relu3
    

    def save(self, save_path='./model.ckpt'):
        saved_path = self.saver.save(self.sess, save_path)
        print("Model saved in file: %s"%saved_path)

    def load(self, load_path = './model.ckpt'):
        self.saver.restore(self.sess, load_path)
        print("Model restored")

    def terminate(self):
        self.sess.close()
        tf.reset_default_graph() 

# 1. Data Load
### 전처리 width, height, input_channel 정의

In [ ]:
X = dataload~
Y = dataload~

In [ ]:
#예시
width = X.shape[0]
height = X.shape[1]
input_channel = X.shape[2]

# 2. Training condition 정의

In [ ]:
t_loss=[]
t_acc=[]

v_loss=[]
v_acc=[]

val_freq = 1
save_freq = 1
num_epochs= 50


std= 0.01
batch_size =128
l_rate = 1e-3

In [37]:
train_history = pd.DataFrame(index=np.arange(0, num_epochs), columns=['epoch', 'loss', 'acc','timestamp'])
valid_history = pd.DataFrame(index=np.arange(0, num_epochs/val_freq),columns=['epoch', 'loss', 'acc','timestamp'])
hot_encoded_target = np.asarray(pd.get_dummies(np.asarray(Y).flatten()))

# 3. model structure 정의

In [ ]:
if 'MC_dropout_model' in globals():
    MC_dropout_model.terminate()
    
    
MC_dropout_model =CNN_MC_dropout(width= width, height=height,input_channel=input_channel ,std=std,batch_size=batch_size,l_rate=l_rate)

In [ ]:
print (MC_dropout_model.input)
print ('[layer1] - MC dropout')
print (MC_dropout_model.conv1)
print (MC_dropout_model.batch1)
print (MC_dropout_model.relu1)
print (MC_dropout_model.dropout1)
print ('[layer2] - MC dropout')
print (MC_dropout_model.conv2)
print (MC_dropout_model.batch2)
print (MC_dropout_model.relu2)
print (MC_dropout_model.dropout2)
print ('[layer3]')
print (MC_dropout_model.conv3)
print (MC_dropout_model.batch3)
print (MC_dropout_model.relu3)

print ('[layer4]')
print (MC_dropout_model.GAP)
print (MC_dropout_model.logits)

# 4. Training

In [ ]:
#dropout을 training시에는 적용시키지 않고, prediction 시에만 적용
#따라서 train에서는 keep prob = 1.0, dropout_bool = False

for epoch in range(0,epoch):
    for batch in getbatch(X,Y):
        train_in , train_target = batch[0],batch[1]
        MC_dropout_model.train(data=train_in,target= train_target, keep_prob=1.0, dropout_bool=False)

# 5. Trained 된 모델에서 Last Conv output Load하기

In [ ]:
# prediction 시에는 MC dropout 을 적용
# keep_prob = 0.5(survive 확률) , dropout_bool=True

#각 데이터 마다 MC dropout 을 B 번 적용한다면, B 개의 서로 다른 model structure를 거치며 나타난 last conv output을 각 노드마다 모두 저장해놓음

#예를들어, unit 1이 보는 pattern을 모두 last_conv_patter1에 저장
#데이터가 N 개이고, MC dropout 횟수가 B번이라면, 총 N*B개의 패턴이 저장됨
last_conv_pattern1=[]
B = MCdropout횟수

for batch in getbatch(X,Y):
    for i in range(0,B):
        output = MC_dropout_model.get_last_conv_output(data=train_in,target= train_target,keep_prob=0.5,dropout_bool=True)
        pattern = output[output>threshold]
        last_conv_pattern1.append(pattern)

# 6. pattern 저장해 놓은 것을 clustering 해서 라벨 붙혀주기?

# 7. 각 노드당 pattern의 entropy 계산하여 uncertainty 계산?

# 8. uncertainty 가 특정 threshold보다 낮을 경우, pattern과 unit 매칭